In [1]:
# generic
import pandas as pd
import os, glob
import numpy as np
import scipy as sp

# geospatial plotting
import cartopy
import cartopy.crs as ccrs 
import cartopy.feature as cfeature

# display
from IPython.display import Image
from cartopy.io.img_tiles import GoogleTiles

# plotting
import matplotlib.pyplot as plt
import seaborn as sns

# handling netCDFs
import xclim as xc
import xarray as xr

/Users/Violet/Desktop/uniB/SAI-impacts/SAIenv/lib/python3.9/site-packages/xarray/backends/cfgrib_.py:29: UserWarning: Failed to load cfgrib - most likely there is a problem accessing the ecCodes library. Try `import cfgrib` to get the full error message
  warnings.warn(


In [2]:
# open a netCDFs downloaded from the curl script
# read in all the netcdfs in a folder, and make a dict with the run number as the key and xarrays values

def parse_data(fp_to_nc_folder):

    # lists to hold the netcdfs and their run numbers 
    run_numbers = []
    nc_names = []

    # read all the files matching the input file path
    for nc_file in glob.glob(fp_to_nc_folder):

        # open them!
        with open(os.path.join(os.getcwd(), nc_file), 'r') as f: 

            # extract the run number from the file name and add it to a list to ID each file
            # this assumes all ARISE-SAI files have more or less the same filename and location of the run number within it, may need to be updated
            run_numbers.append(pd.Series(nc_file).str.split(pat = 'DEFAULT.')[0][1].split('.cam')[0])

            # open each nc file and add that to another list
            nc_names.append(xr.open_dataset(nc_file))

    # combine the lists into a dict ordered by keys (run numbers low to high)
    data_dict = dict(sorted(dict(zip(run_numbers, nc_names)).items()))

    return(data_dict)

# use this function on the TMSO2 data
TMSO2_dict = parse_data('./project-data/TMSO2/*.nc')

/Users/Violet/Desktop/uniB/SAI-impacts/SAIenv/lib/python3.9/site-packages/xarray/backends/plugins.py:71: RuntimeWarning: Engine 'cfgrib' loading failed:
Cannot find the ecCodes library
  warnings.warn(f"Engine {name!r} loading failed:\n{ex}", RuntimeWarning)


In [3]:
TMSO2_dict['001']

<xarray.Dataset>
Dimensions:       (lat: 192, zlon: 1, nbnd: 2, lon: 288, lev: 70, ilev: 71,
                   time: 420)
Coordinates:
  * lat           (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * zlon          (zlon) float64 0.0
  * lon           (lon) float64 0.0 1.25 2.5 3.75 ... 355.0 356.2 357.5 358.8
  * lev           (lev) float64 5.96e-06 9.827e-06 1.62e-05 ... 976.3 992.6
  * ilev          (ilev) float64 4.5e-06 7.42e-06 1.223e-05 ... 985.1 1e+03
  * time          (time) object 2035-02-01 00:00:00 ... 2070-01-01 00:00:00
Dimensions without coordinates: nbnd
Data variables: (12/27)
    zlon_bnds     (zlon, nbnd) float64 ...
    gw            (lat) float64 ...
    hyam          (lev) float64 ...
    hybm          (lev) float64 ...
    P0            float64 ...
    hyai          (ilev) float64 ...
    ...            ...
    n2ovmr        (time) float64 ...
    f11vmr        (time) float64 ...
    f12vmr        (time) float64 ...
    sol_tsi       (time) float64 ...
    nsteph        (time) int32 ...
    TMSO2         (time, lat, lon) float32 ...
Attributes:
    Conventions:       CF-1.0
    source:            CAM
    case:              b.e21.BW.f09_g17.SSP245-TSMLT-GAUSS-DEFAULT.001
    logname:           geostrat
    host:              cheyenne5
    initial_file:      b.e21.BWSSP245cmip6.f09_g17.CMIP6-SSP2-4.5-WACCM.001.c...
    topography_file:   /glade/p/cesmdata/cseg/inputdata/atm/cam/topo/fv_0.9x1...
    model_doi_url:     https://doi.org/10.5065/D67H1H0V
    time_period_freq:  month_1

In [4]:
# make a function that plots the variable of interest around the globe using cartopy
projection=ccrs.PlateCarree()
bbox=[-180,180,-60,85];creg='glob'
# mplt.rc('xtick', labelsize=9) 
# mplt.rc('ytick', labelsize=9)

nc = TMSO2_dict['001']
lat = nc.variables['lat'][:]
lon = nc.variables['lon'][:]
kopi = (nc.variables['TMSO2'][:,:])
nc.close()

fig = plt.figure(figsize=(11,5))
ax = fig.add_subplot(1,1,1,projection=projection)
ax.set_extent(bbox,projection)
ax.add_feature(cfea.COASTLINE,lw=.5)
ax.add_feature(cfea.RIVERS,lw=.5)
ax.add_feature(cfea.BORDERS, linewidth=0.6, edgecolor='dimgray')
# ax.background_patch.set_facecolor('.9')
# levels = [1,4,8,11,14,17,21,25,29]
# cmap = plt.cm.BrBG
# norm = mplt.colors.BoundaryNorm(levels,cmap.N)
# ddlalo = .25
# pc = ax.contourf(lon,lat,kopi,levels=levels,transform=projection,cmap=cmap,norm=norm,extend='both')
# divider = make_axes_locatable(ax)
# ax_cb = divider.new_horizontal(size="3%", pad=0.1, axes_class=plt.Axes)

# fig.colorbar(pc,extend='both', cax=ax_cb)
# fig.add_axes(ax_cb)
# fig.colorbar(pc,extend='both', cax=ax_cb)
# ttitle='Jony'
# ax.set_title(ttitle,loc='left',fontsize=9)
plt.show()

In [ ]:
# The first five members of ARISE-SAI-1.5 simulations were initialized in 2035 from the first five members (001 to 005) 
# of the SSP2-45 simulations carried out with CESM2(WACCM6); hence, all had different initial ocean, sea-ice, land, 
# and atmospheric initial conditions on January 1, 2035. Similarly to the SSP2-45 simulations, subsequent ensemble members 
# (006 through 010) were initialized from the same initial conditions as members 001 through 005, respectively, 
# with an addition of a small temperature perturbation to the atmospheric initial condition to create ensemble spread.
# ([6, 7, 8, 9, 10] x 10ˆ-14 K, respectively)
print(TMSO2_dict['001'])

In [ ]:
# parse the correct ne python -m pip install "xarray[io]"tCDF for so4_a1DDF
# try to get a time seires for deposition
# compare this to some other sulfur fluxes
# see if this is high, low, etc.